# Linneus Project

A project where I can detect flowers given an image. Trying to put create my own CNN.

~ Tafoya

In [2]:
import os
from PIL import Image
import numpy as np
import random
import json

flower_types = []
try :
    with open('./archive/cat_to_name.json', 'r', encoding='utf-8') as file:
        flower_types = json.load(file)
except FileNotFoundError:
    print("File not found")

directory = "archive"
valid = ('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')

all_images = []
all_labels = []

print(flower_types)

{'21': 'fire lily', '3': 'canterbury bells', '45': 'bolero deep blue', '1': 'pink primrose', '34': 'mexican aster', '27': 'prince of wales feathers', '7': 'moon orchid', '16': 'globe-flower', '25': 'grape hyacinth', '26': 'corn poppy', '79': 'toad lily', '39': 'siam tulip', '24': 'red ginger', '67': 'spring crocus', '35': 'alpine sea holly', '32': 'garden phlox', '10': 'globe thistle', '6': 'tiger lily', '93': 'ball moss', '33': 'love in the mist', '9': 'monkshood', '102': 'blackberry lily', '14': 'spear thistle', '19': 'balloon flower', '100': 'blanket flower', '13': 'king protea', '49': 'oxeye daisy', '15': 'yellow iris', '61': 'cautleya spicata', '31': 'carnation', '64': 'silverbush', '68': 'bearded iris', '63': 'black-eyed susan', '69': 'windflower', '62': 'japanese anemone', '20': 'giant white arum lily', '38': 'great masterwort', '4': 'sweet pea', '86': 'tree mallow', '101': 'trumpet creeper', '42': 'daffodil', '22': 'pincushion flower', '2': 'hard-leaved pocket orchid', '54': 's

In [3]:
# Kernels
"""
kernels = []

for k in range(32):
    kernel_3d = []
    for row in range(3):
        kernel_row = []
        for col in range(3):
            pixel = [random.uniform(-1.0, 1.0),
                    random.uniform(-1.0, 1.0),
                    random.uniform(-1.0, 1.0)]
            kernel_row.append(pixel)
        kernel_3d.append(kernel_row)
    kernels.append(kernel_3d)
"""
kernels1 = np.random.uniform(-1.0, 1.0, (32, 3, 3, 3))

In [4]:
def converting_to_matrix(flower, target_size=(128, 128)):
    folder_path = os.path.join(directory, "train", flower)

    if not os.path.isdir(folder_path):
        print(f"Warning: {folder_path} not found")
        return

    label = flower

    for file in os.listdir(folder_path):
        if file.endswith(valid):
            img_path = os.path.join(folder_path, file)
            try:
                with Image.open(img_path) as img:
                    img = img.convert('RGB')
                    img = img.resize(target_size)
                    all_images.append(np.array(img))
                    all_labels.append(label)

            except Exception as e:
                print(f"Error loading {file}: {e}")


In [5]:
# Reading images

if not os.path.isdir(directory):
    print("Directory doesn't exist")
    exit(0)

for flower in flower_types:
    converting_to_matrix(flower)

all_images = np.array(all_images)
all_labels = np.array(all_labels)

print(len(all_images))
print(all_images[0].shape)


6552
(128, 128, 3)


In [6]:
unique, counts = np.unique(all_labels, return_counts=True)
for label, count in zip(unique, counts):
    flower_name = flower_types[label]
    print(f"  {flower_name} (label {label}): {count} images")
print()

for i in range(min(5, len(all_images))):
    print(f"  Image {i}: shape={all_images[i].shape}, label={all_labels[i]} ({flower_types[all_labels[i]]})")
print()

  pink primrose (label 1): 27 images
  globe thistle (label 10): 38 images
  blanket flower (label 100): 35 images
  trumpet creeper (label 101): 49 images
  blackberry lily (label 102): 36 images
  snapdragon (label 11): 68 images
  colt's foot (label 12): 73 images
  king protea (label 13): 38 images
  spear thistle (label 14): 44 images
  yellow iris (label 15): 38 images
  globe-flower (label 16): 36 images
  purple coneflower (label 17): 60 images
  peruvian lily (label 18): 65 images
  balloon flower (label 19): 38 images
  hard-leaved pocket orchid (label 2): 49 images
  giant white arum lily (label 20): 46 images
  fire lily (label 21): 34 images
  pincushion flower (label 22): 47 images
  fritillary (label 23): 72 images
  red ginger (label 24): 35 images
  grape hyacinth (label 25): 34 images
  corn poppy (label 26): 33 images
  prince of wales feathers (label 27): 36 images
  stemless gentian (label 28): 55 images
  artichoke (label 29): 62 images
  canterbury bells (label 3

In [7]:
# Shuffling the dataset
indices = np.arange(len(all_images))
np.random.shuffle(indices)

all_images = all_images[indices]
all_labels = all_labels[indices]

# Let's normalize the dataset
flowers = all_images.astype('float32') / 255.0

In [8]:
# Applying convolution

# Very slow
"""
def convolution_slow(images):
    images_convolution = []
    for img in images:
        feature_maps = []
        for kernel in kernels:
            feature_map = []

            for rows in range(0, len(img) - 3 + 1):
                row_output = []
                for cols in range(0, len(img) - 3 + 1):
                    dot_sum = 0
                    for kernel_i in range(0, 3):
                        for kernel_j in range(0, 3):
                            for layer in range(0, 3):
                                dot_sum += img[rows + kernel_i][cols + kernel_j][layer] * kernel[kernel_i][kernel_j][layer]
                    row_output.append(dot_sum)
                feature_map.append(row_output)

            feature_maps.append(feature_map)

        images_convolution.append(feature_maps)

    return images_convolution
"""

'\ndef convolution_slow(images):\n    images_convolution = []\n    for img in images:\n        feature_maps = []\n        for kernel in kernels:\n            feature_map = []\n\n            for rows in range(0, len(img) - 3 + 1):\n                row_output = []\n                for cols in range(0, len(img) - 3 + 1):\n                    dot_sum = 0\n                    for kernel_i in range(0, 3):\n                        for kernel_j in range(0, 3):\n                            for layer in range(0, 3):\n                                dot_sum += img[rows + kernel_i][cols + kernel_j][layer] * kernel[kernel_i][kernel_j][layer]\n                    row_output.append(dot_sum)\n                feature_map.append(row_output)\n\n            feature_maps.append(feature_map)\n\n        images_convolution.append(feature_maps)\n\n    return images_convolution\n'

In [9]:
# ReLu

# Also slow
"""
def relu_slow(images):
    for img in images:
        for feature_map_idx in range(len(img)):  # 32 feature maps
            for row in range(len(img[feature_map_idx])):  # 126 rows
                for col in range(len(img[feature_map_idx][row])):  # 126 cols
                    # FIXED: No layer loop! Each value is just a number, not RGB
                    img[feature_map_idx][row][col] = max(0, img[feature_map_idx][row][col])

    return images
"""

'\ndef relu_slow(images):\n    for img in images:\n        for feature_map_idx in range(len(img)):  # 32 feature maps\n            for row in range(len(img[feature_map_idx])):  # 126 rows\n                for col in range(len(img[feature_map_idx][row])):  # 126 cols\n                    # FIXED: No layer loop! Each value is just a number, not RGB\n                    img[feature_map_idx][row][col] = max(0, img[feature_map_idx][row][col])\n\n    return images\n'

In [10]:
# Pooling

# Slow
"""
def pool_slow(images):
    images_pooling = []

    for img in images:
        pooled_feature_maps = []

        for feature_map in img:  # For each of 32 feature maps
            pooled_map = []

            # Use step=2 (stride) and divide size by 2
            for rows in range(0, len(feature_map), 2):  # Step by 2
                pooled_row = []
                for cols in range(0, len(feature_map[0]), 2):  # Step by 2

                    # Find max in 2×2 window
                    mx = float('-inf')  # Start with very small number
                    for pool_i in range(0, 2):
                        for pool_j in range(0, 2):
                            if rows + pool_i < len(feature_map) and cols + pool_j < len(feature_map[0]):
                                mx = max(mx, feature_map[rows + pool_i][cols + pool_j])

                    pooled_row.append(mx)
                pooled_map.append(pooled_row)

            pooled_feature_maps.append(pooled_map)

        images_pooling.append(pooled_feature_maps)

    return images_pooling
"""

"\ndef pool_slow(images):\n    images_pooling = []\n\n    for img in images:\n        pooled_feature_maps = []\n\n        for feature_map in img:  # For each of 32 feature maps\n            pooled_map = []\n\n            # Use step=2 (stride) and divide size by 2\n            for rows in range(0, len(feature_map), 2):  # Step by 2\n                pooled_row = []\n                for cols in range(0, len(feature_map[0]), 2):  # Step by 2\n\n                    # Find max in 2×2 window\n                    mx = float('-inf')  # Start with very small number\n                    for pool_i in range(0, 2):\n                        for pool_j in range(0, 2):\n                            if rows + pool_i < len(feature_map) and cols + pool_j < len(feature_map[0]):\n                                mx = max(mx, feature_map[rows + pool_i][cols + pool_j])\n\n                    pooled_row.append(mx)\n                pooled_map.append(pooled_row)\n\n            pooled_feature_maps.append(pooled_ma

In [11]:
def convolution(images, kernels) :
    n = images.shape[0]
    output_size = images.shape[1] - 3 + 1  # 126
    num_kernels = kernels.shape[0]  # 32

    output = np.zeros((n, num_kernels, output_size, output_size))

    for img_idx in range(n):
        for kernel_idx in range(num_kernels):
            for idx in range(output_size):
                for j in range(output_size):
                    patch = images[img_idx, idx:idx + 3, j:j + 3, :]

                    output[img_idx, kernel_idx, idx, j] = np.sum(patch * kernels[kernel_idx])

    return output

In [12]:
def relu(images) :
    return np.maximum(0, images)

In [13]:
def pooling(images, pool_size=2):

    n, num_features, height, width = images.shape

    new_height = height // pool_size
    new_width = width // pool_size

    output = np.zeros((n, num_features, new_height, new_width))

    for img_idx in range(n):
        for feature_idx in range(num_features):
            for idx in range(new_height):
                for j in range(new_width):
                    window = images[img_idx, feature_idx,
                                idx * pool_size:(idx + 1) * pool_size,
                                j*pool_size:(j+1)*pool_size]

                    output[img_idx, feature_idx, idx, j] = np.max(window)

    return output

In [14]:
def flatten(images):
    """
    Flatten 4D tensor to 2D matrix
    Input: (N, channels, height, width)
    Output: (N, channels*height*width)
    """
    n = images.shape[0]
    flattened_size = np.prod(images.shape[1:])  # multiply all dimensions except batch
    return images.reshape(n, flattened_size)



In [16]:
# We will make 2 convolutions
conv1 = convolution(all_images, kernels1)
relu1 = relu(conv1)
pool1 = pooling(relu1)
# IMPORTANT: Need to transpose from (N, 32, 63, 63) to (N, 63, 63, 32)
# because our convolution expects channels as last dimension
pool1_transposed = np.transpose(pool1, (0, 2, 3, 1))

kernels2 = np.random.uniform(-1.0, 1.0, (64, 3, 3, 32))
conv2 = convolution(pool1_transposed, kernels2)
relu2 = relu(conv2)
pool2 = pooling(relu2)

output = flatten(pool2)

(6552, 128, 128, 3)


KeyboardInterrupt: 

In [ ]:
# Start of NN

class NeuralNetwork:
    def __init__(self, layers):
        """
            layers it's a list giving the dimensions of each layer
            ex: [128, 64, 32, 16]
        """
        self.layers = layers
        self.num_layers = len(layers)

        self.weights = []
        self.biases = []

        for i in range(self.num_layers - 1):
            w = np.random.randn(layers[i], layers[i+1]) * np.sqrt(2.0 / layers[i])
            self.weights.append(w)

            b = np.zeros((1, layers[i+1]))
            self.biases.append(b)

        def forward(given_input):
            output = np.zeros(layers[1])

            n = len(layers[1])
            for i in range(n):
                for j in range()

